<a href="https://colab.research.google.com/github/rrrudolph/trade/blob/master/Strategy_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Format Incoming Data


In [87]:

import pandas as pd
import datetime as dt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('drive/My Drive/Colab Notebooks/Data/EURUSD15.csv')
df = df[1200:1350]

#### This stuff was to format an original MT4 OHLC file ####

# df = df.set_axis(['Date', 'Time','O','H','L','C','V'], axis=1, inplace=False)
# df['DT'] = df['Date'] + ' '+df['Time']
df['DT'] = pd.to_datetime(df['DT'])
# df.drop(columns=['Date','Time'],inplace=True)
df = df.reindex(columns=['DT', 'O', 'H', 'L', 'C', 'V', 'Bar', 'D_Range',
                         'ADR', 'Pct_Change', 'Frac_H', 'Frac_L', 'Locked_H', 'Locked_L',
                         'Sw_Price','Sw_Rating', 'Sw_Size', 'Sw_Pct', 'Sw_Weight',
                         'TZ_Active', 'TZ_Type', 'TZ_Start',
                         'TZ_End', 'TZ_H', 'TZ_L'])

# df['DT'] -= pd.Timedelta(hours=7) # minus 7 hours to match CST

# set bar type
df['Bar'][df['C'] < df['O']] = 'down'
df['Bar'][df['C'] > df['O']] = 'up'


# if you want to re-iterate over the swings uncomment these:
df['Locked_H'] = 0
df['Locked_L'] = 0

df.reset_index(drop=True,inplace=True)
df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,Sw_Weight,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L
0,2017-08-15 14:15:00,1.17556,1.17566,1.17451,1.17482,1162,down,0.01055,0.00899,NaN,0.0,0.0,0,0,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-15 14:30:00,1.17480,1.17480,1.17383,1.17393,1245,down,0.01055,0.00900,NaN,0.0,0.0,0,0,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-15 14:45:00,1.17392,1.17410,1.17337,1.17398,1085,up,0.01055,0.00900,NaN,0.0,2.0,0,0,NaN,1.0,0.00111,0.12,0.999846,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-15 15:00:00,1.17398,1.17448,1.17359,1.17421,1419,up,0.01055,0.00901,NaN,0.0,0.0,0,0,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-15 15:15:00,1.17422,1.17448,1.17362,1.17390,1184,down,0.01055,0.00901,NaN,2.0,0.0,0,0,NaN,1.0,-0.00576,-0.64,0.998539,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2017-08-17 02:30:00,1.17719,1.17738,1.17696,1.17711,244,down,0.01277,0.00906,NaN,0.0,2.0,0,0,NaN,1.0,0.00045,0.05,0.999940,NaN,NaN,NaN,NaN,NaN,NaN
146,2017-08-17 02:45:00,1.17710,1.17741,1.17709,1.17735,262,up,0.01277,0.00907,NaN,0.0,0.0,0,0,NaN,NaN,0.00000,0.00,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
147,2017-08-17 03:00:00,1.17736,1.17745,1.17702,1.17716,579,down,0.01277,0.00908,NaN,3.0,0.0,0,0,NaN,2.0,-0.00097,-0.11,0.999861,NaN,NaN,NaN,NaN,NaN,NaN
148,2017-08-17 03:15:00,1.17716,1.17716,1.17648,1.17678,654,down,0.01277,0.00908,NaN,0.0,19.0,0,0,NaN,10.0,0.00250,0.28,0.999576,NaN,NaN,NaN,NaN,NaN,NaN


# ADR

In [0]:

#### get the D_Range and ADR  ####

# To calculate this per day it needs to lag by 24 hrs 
# Otherwise the current day will always look small
# better to just reference a daily df

temp = df[df['DT'] < dt.datetime.today()]  # return all days prior to today (for live data) 
for i in temp['DT'].dt.date.unique():   # iterate through each unique day
    group = df.groupby(df['DT'].dt.date).get_group(i)   # group data by day
    dayrange = max(group['H']) - min(group['L'])   
    df.loc[group.index,'D_Range'] = dayrange

# ADR
#  There are 96 15m candles per day and I want a 5 day average.  
#  This will give me an index error so it needs a try block
##### NOTE: this needs to be modified so it can remain a 5 day
#####       average given any timeframe
for i in temp.index:
    try:
        df.loc[i,'ADR'] = round(df.loc[i-480:i,'D_Range'].mean(),5)
    except:
        num_rows = df.iloc[i].index - df.iloc[0].index
        df.loc[i,'ADR'] = round(df.loc[i - num_rows:i,'D_Range'].mean(),5)

# lastly, fill today's blank ADR values with yesterdays
df['ADR'].fillna(method='ffill', inplace=True)


####  Set Pct_Change  ####
df['Pct_Change'] = df['C'].pct_change()

# Fractals

In [0]:
highs = df['H'].values
lows = df['L'].values

# ommitt rows where the swings are locked
temp = df[(df['Locked_H'] != 1) & (df['Locked_L'] != 1)]


# iterate over the whole df using the unlocked swings
# as starting points in the loop 
for i in temp.index:
  low = lows[i]
  high = highs[i]
 
  # the order below is highs forwards then highs backwards

  try:   # iter forwards 
    count_prior = 0   # initialize counters and such
    count_next = 0
    next_ = i + 1
    prior_ = i - 1

    while high > highs[next_] and count_next < 80: # iter limit
      next_ +=1
      count_next +=1
    df.loc[i,'Locked_H'] = 1 # if it hasn't error'd lock the row 
  except:
    pass

  try: # iter backwards
    while high >= highs[prior_] and count_prior < 80: 
      prior_ -=1
      count_prior +=1
  except:
    pass 
  df.loc[i,'Frac_H'] = min([count_prior, count_next])
  try: # the 'min is an empty argument' requires try block
    if df.loc[i,'Frac_H'] > 1:  # this will return the swing count if frac > 1
      df.loc[i,'Swing'] = min(lows[i+1:i+df.loc[i,'Frac_H']]) - high

  except:
    df.loc[i,'Swing'] = 0

  # now the same thing but for the lows

  try: # iter forwards 
    count_prior = 0
    count_next = 0
    next_ = i + 1
    prior_ = i - 1

    while low < lows[next_] and count_next < 80: 
      next_ +=1
      count_next +=1
    df.loc[i,'Locked_L'] = 1 # if it hasn't error'd set the lock value 
  except:
    pass

  try: # iter backwards
    while low <= lows[prior_] and count_prior < 80: 
      prior_ -=1
      count_prior +=1  
  except:
    pass
  df.loc[i,'Frac_L'] = min([count_prior, count_next])

  # find swing size (price movement from hi to low or low to hi)
  # only overwrite if the new value is greater 
  try:  
    if (max(highs[i:i+count_next]) - low > abs(df.loc[i,'Swing'])
        and df.loc[i,'Frac_L'] > 1):
      df.loc[i,'Swing'] = max(highs[i+1:df.loc[i,'Frac_L']]) - low
  except:
      pass



#### Fractals viz

In [90]:
fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]

ratings = round(df['Frac_H'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Frac_L'][df['Sw_Size'] > 0],0) 

fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove this "xaxis" section to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)

# Swings

In [97]:


#### Calculate the Swing Ratings #### 

# get the swing size in relation to ADR
df['Sw_Pct'] = round(df['Sw_Size']/df['ADR'], 2)

# create a function of a curved line that will act as a weight
# to use on the swing size. this will taper off the extreme
# swing sizes to balance out the swing ratings overall
df['Sw_Weight'] = -1.02 ** abs(df['Sw_Pct']*100) # function for curved line
weight = df['Sw_Weight'][abs(df['Sw_Weight'].values) > 0] # list of weights 

# first fill NaNs with mean of group
# normalize so its a weight between 0 and 1
# now that the weights are normalized, multiply the swing by its weight
# add the fractal rating to its swing and divide by 2

weight.fillna(value=weight.mean(), inplace=True) 
df['Sw_Weight'] = (weight - min(weight)) / (max(weight) - min(weight))
final = df['Sw_Size'] * df['Sw_Weight']
temp = df[abs(df['Sw_Size']) > 0]
df['Sw_Rating'] = round((abs(final) * 2000 + temp['Frac_H'] + temp['Frac_L']) / 2)



df.drop(columns=['Sw_Weight'], inplace=True)

# save the swing prices
for i in df.index:
    a = df.loc[i,'H'][ df.loc[i,'Sw_Size'] < 0]
    b = df.loc[i,'L'][ df.loc[i,'Sw_Size'] > 0]
    if a.size > 0:
        df.loc[i,'Sw_Price'] = a
    elif b.size > 0: 
        df.loc[i,'Sw_Price'] = b

df.head(20)

,DT,O,H,L,C,V,Bar,D_Range,ADR,Pct_Change,Frac_H,Frac_L,Locked_H,Locked_L,Sw_Price,Sw_Rating,Sw_Size,Sw_Pct,TZ_Active,TZ_Type,TZ_Start,TZ_End,TZ_H,TZ_L,Swing,Impulse
0,2017-08-15 14:15:00,1.17556,1.17566,1.17451,1.17482,1162,down,0.00694,0.00694,NaN,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,2017-08-15 14:30:00,1.17480,1.17480,1.17383,1.17393,1245,down,0.00694,0.00694,-0.000758,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2,2017-08-15 14:45:00,1.17392,1.17410,1.17337,1.17398,1085,up,0.00694,0.00694,0.000043,0.0,2.0,1,1,1.17337,2.0,0.00111,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.000000
3,2017-08-15 15:00:00,1.17398,1.17448,1.17359,1.17421,1419,up,0.00694,0.00694,0.000196,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
4,2017-08-15 15:15:00,1.17422,1.17448,1.17362,1.17390,1184,down,0.00694,0.00694,-0.000264,2.0,0.0,1,1,1.17448,5.0,-0.00576,-0.83,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-inf
5,2017-08-15 15:30:00,1.17388,1.17388,1.16935,1.17006,3308,down,0.00694,0.00694,-0.003271,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
6,2017-08-15 15:45:00,1.17003,1.17081,1.16872,1.16918,2613,down,0.00694,0.00694,-0.000752,0.0,52.0,1,1,1.16872,29.0,0.00707,1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196.153846
7,2017-08-15 16:00:00,1.16918,1.17031,1.16900,1.16991,1903,up,0.00694,0.00694,0.000624,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
8,2017-08-15 16:15:00,1.16990,1.17142,1.16979,1.17106,1612,up,0.00694,0.00694,0.000983,0.0,0.0,1,1,NaN,NaN,0.00000,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
9,2017-08-15 16:30:00,1.17108,1.17242,1.17071,1.17202,2240,up,0.00694,0.00694,0.000820,3.0,0.0,1,1,1.17242,4.0,-0.00233,-0.34,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-inf


#### Swings viz

In [92]:
fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]

ratings = round(df['Sw_Rating'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Sw_Rating'][df['Sw_Size'] > 0],0) 

fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove this "xaxis" section to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


#### test adding impulse 

In [100]:
# I also want to calculate the impulse
# of the swing (sw_size over time) and add
# that to the rating formula
df['Impulse'] = 0
temp = df[df['Sw_Rating'] > 0]
for i in temp.index:
    if df.loc[i, 'Sw_Pct'] < 0:
        fractal = df.loc[i, 'Frac_H']
    if df.loc[i, 'Sw_Pct'] > 0:
        fractal = df.loc[i, 'Frac_L']
    print(fractal)
    df.loc[i, 'Impulse'] = df.loc[i, 'Sw_Pct']*10000 / fractal
    # print(df.loc[i, 'Impulse'])

#plot
fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])
# swings = df['Sw_Rating'][df['Sw_Size'] < 0].append(df['Sw_Rating'][df['Sw_Size'] < 0])
hix = df['DT'][df['Sw_Size'] < 0]
lox = df['DT'][df['Sw_Size'] > 0]
hi = df['H'][df['Sw_Size'] < 0]
low = df['L'][df['Sw_Size'] > 0]

ratings = round(df['Impulse'][df['Sw_Size'] < 0],0)
ratingsl = round(df['Impulse'][df['Sw_Size'] > 0],0) 

fig.add_trace(go.Scatter(
    x=hix,
    y=hi,
    mode="markers+text",
    name="Swing Ratings",
    text=ratings,
    textposition="top center"
))

fig.add_trace(go.Scatter(
    x=lox,
    y=low,
    mode="markers+text",
    name="Swing Ratings",
    text=ratingsl.values,
    textposition="bottom center"
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove this "xaxis" section to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)

2.0
2.0
52.0
3.0
2.0
4.0
19.0
11.0
7.0
6.0
2.0
2.0
15.0
3.0
4.0
17.0
10.0
3.0
6.0
2.0
2.0
7.0
5.0
41.0
19.0
2.0
5.0
8.0
2.0
3.0
9.0
46.0
8.0
2.0
13.0
17.0
4.0
2.0
4.0
6.0
2.0
2.0
1.0


In [94]:
stop

NameError: ignored

# Trades and Trade Zone Viz

In [102]:


active_zones = df[(df['TZ_H'] != np.nan) & # there is a zone
                  (df['TZ_Active'] != False)] 

df['TZ_Active'] = np.nan
df['TZ_Type'] = np.nan
df['TZ_Start'] = np.nan
df['TZ_End'] = np.nan

# save zones for vis
plot_buyx = [] 
plot_buyy = []
plot_buy_flipx = [] 
plot_buy_flipy = []
plot_sellx = [] 
plot_selly = []
plot_buy_flipx = [] 
plot_sell_flipy = []
trade_entries = []
dates = []
names = []

temp =  df[(df['TZ_Active'] != False) &
           (df['Sw_Rating'] > 10)]

for i in temp.index: 

    swing = abs(df.loc[i, 'Sw_Size'])
    zone_height = df.loc[i, 'Sw_Rating'] * df.loc[i,'ADR'] / 75

    # set a limit on zone height
    a = df.loc[2, 'DT'] - df.loc[1, 'DT']  
    b = pd.Timedelta(days = 1)
    cpd = b / a         # candles per day
    max_size = df.loc[i, 'ADR'] / cpd * 42  # wont work for D1 TF 

    if zone_height > max_size:
        zone_height = max_size
    # print("zone height: ", zone_height, "\n max: ", max_size)
    if df.loc[i, 'Sw_Size'] < 0:       # swing high
        df.loc[i,'TZ_Type'] = 'sell'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price'] + zone_height     # upper price 
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price']     # lower    
    if df.loc[i, 'Sw_Pct'] > 0:      # swing low
        df.loc[i,'TZ_Type'] = 'buy'
        df.loc[i,'TZ_H'] = df.loc[i,'Sw_Price']     
        df.loc[i,'TZ_L'] = df.loc[i,'Sw_Price'] - zone_height    
    
    ####  set zone end  ####
    end = round(df.loc[i, 'Sw_Rating'] * 3.5 + i)   # now get the length of the zone
    if end >  len(df) - 1:      # set max value as the length of df
        end =  len(df) - 1
    # elif end > round(max_size * 10000):
    #     end = round(max_size * 10000)
    df.loc[i,'TZ_End'] = end
   
    ####  set a custom start  #### (makes viz difficult)
    # start = i  + round((end - i) * 0.2) 
    # if start >  len(df) - 1:
    #     start =  len(df) - 1
    df.loc[i,'TZ_Start'] = i #start

    ####  set buffers  ####  (used for switching zone type)
    df['Buf_Hi'] = df['TZ_H'] + (zone_height * 0.25)
    df['Buf_Lo'] = df['TZ_L'] - (zone_height * 0.25)

    ####  df of candles within current zone  ####
    zone_df = df.loc[start:end]
    zone_df = zone_df[(zone_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                      (zone_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
   
#    for backtesting data where I feed in a whole df, I need to slice 
#    the zones up into parts in order to capture the appropriate trades
#    that would have happened in a bar by bar, live environment
#    to account for breached zones

    if df.loc[i,'TZ_Type'] == 'sell':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles
        # but first see if zone gets breached, and if so, shorten the zone_df
        # to end where the first breach happens

            # check if zone gets breached to the upside
            new_high = zone_df['H'][zone_df['H'] > df.loc[i, 'Buf_Hi']]
            if len(new_high) > 0: 
                end = min(new_high.index)     # set end of slice at breach
                zone_df = zone_df.iloc[start:end]

            # low volume up bar
            signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'up')]
            if len(signal_a) > 0:
                # in case of multiple signals, return highest entry price
                sell_price = max(signal_a['O'])
                idx = signal_a['O'].idxmax() # get index of entry candle
                date = zone_df.loc[idx, 'DT']

                # plot first slice
                trade_entries.append(sell_price)
                dates.append(date)
                names.append('sell')
                plot_sellx.append(df.loc[start,'DT'])
                plot_sellx.append(df.loc[end,'DT'])
                plot_sellx.append(df.loc[end,'DT'])
                plot_sellx.append(df.loc[start,'DT'])
                plot_sellx.append(df.loc[start,'DT'])
                plot_sellx.append(None)
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(df.loc[i,'TZ_H'])
                plot_selly.append(df.loc[i,'TZ_H'])
                plot_selly.append(df.loc[i,'TZ_L'])
                plot_selly.append(None)

                # now end becomes start (second slice)
                if len(new_high) > 0: 
                    # change zone type and look for trades in new zone_df slice
                    df.loc[i,'TZ_Type'] = 'buy' # change zone type
                    start = min(new_high.index)    # this is the 'end' of first slice
                    end = round((end - start) * 0.5 + end)     # shift end forward by 50%
                    if end > len(df) - 1:
                        end = len(df) - 1
                    df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 # shift zone down by 50%
                    df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

                    # reset the zone_df to reflect new zone dimensions
                    flipped_df = df.loc[start:end]
                    flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                    (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]
            
            

                    # and look for trades again, but this time buys. if zone
                    # gets breached for a second time it becomes inactive   

                    new_low = flipped_df['L'][flipped_df['L'] < df.loc[i, 'TZ_L']]   
                    if len(flipped_df) > 0:     # first see if any candles exist in new zone
                        if len(new_low) > 0:    # then see if it got breached 
                            idx = min(new_low.index)    # irst candle to breach
                            df.loc[i,'TZ_Active'] = False  
                            # shorten the flipped_df to end at that candle
                            # (this won't be relevent for live data, but it's necessary
                            #  to backtest like this. for live data, once the zone
                            #  becomes inactive it will simply be ignored and any
                            #  trades by that point would have already been found)
                            flipped_df = flipped_df.loc[start:idx]
                        signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'down')]
                        if len(signal_a) > 0:
                            # in case of multiple signals, get lowest price for buy 
                            buy_price = min(signal_a['O'])
                            idx = signal_a['O'].idxmin()  
                            date = df.loc[idx, 'DT']
                            
                            # plot 
                            trade_entries.append(buy_price)
                            dates.append(date)
                            names.append('buy')            
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(df.loc[end, 'DT'])
                            plot_sell_flipx.append(df.loc[end, 'DT'])
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(df.loc[start, 'DT'])
                            plot_sell_flipx.append(None)
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_H'])
                            plot_sell_flipy.append(df.loc[i,'TZ_L'])
                            plot_sell_flipy.append(None)
                    
    if df.loc[i,'TZ_Type'] == 'buy':
        if len(zone_df) > 0:
        # if candles are found, look for a candle with lower vol than 
        # original swing, or a series of 3 sequentially lower vol candles


            # check if zone gets breached to the downside
            new_low = zone_df['L'][zone_df['L'] < df.loc[i, 'Buf_Lo']]
            if len(new_low) > 0: 
                end = min(new_low.index)     # set end of slice at breach
                zone_df = zone_df.loc[start:end]

            signal_a = zone_df[(zone_df['V'] < df.loc[i,'V']) & (zone_df['Bar'] == 'down')]
            if len(signal_a) > 0:
                # in case of multiple signals, return highest entry price
                buy_price = min(signal_a['O'])
                idx = signal_a['O'].idxmin() # get index of entry candle
                date = zone_df.loc[idx, 'DT']

                # plot 
                trade_entries.append(buy_price)
                dates.append(date)
                names.append('buy')
                plot_buyx.append(df.loc[start,'DT'])
                plot_buyx.append(df.loc[end,'DT'])
                plot_buyx.append(df.loc[end,'DT'])
                plot_buyx.append(df.loc[start,'DT'])
                plot_buyx.append(df.loc[start,'DT'])
                plot_buyx.append(None)
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(df.loc[i,'TZ_H'])
                plot_buyy.append(df.loc[i,'TZ_H'])
                plot_buyy.append(df.loc[i,'TZ_L'])
                plot_buyy.append(None)

            # now work the second slice
            if len(new_low) > 0: 
                df.loc[i,'TZ_Type'] = 'sell' # change zone type
                start = min(new_low.index)    # shift start forward to first of breach candles
                end = round((end - start) * 0.5 + end)     # shift end forward by 50%
                if end > len(df) - 1:
                    end = len(df) - 1
                df.loc[i,'TZ_H'] = df.loc[i,'TZ_H'] - zone_height / 2 # shift zone down by 50%
                df.loc[i,'TZ_L'] = df.loc[i,'TZ_L'] - zone_height / 2 

                # reset the zone_df to reflect new zone dimensions
                flipped_df = df.loc[start:end]
                flipped_df = flipped_df[(flipped_df['H'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H'])) |
                                (flipped_df['L'].between(df.loc[i,'TZ_L'], df.loc[i,'TZ_H']))]

                # and look for trades again, but this time buys. if zone
                # gets breached for a second time it becomes inactive   

                new_high = flipped_df['H'][flipped_df['H'] > df.loc[i, 'TZ_H']]   
                if len(flipped_df) > 0:     # first see if any candles exist in new zone
                    if len(new_high) > 0:    # then see if it got breached 
                        idx = min(new_high.index)    # irst candle to breach
                        df.loc[i,'TZ_Active'] = False  
                        # shorten the flipped_df to end at that candle
                        # (this won't be relevent for live data, but it's necessary
                        #  to backtest like this. for live data, once the zone
                        #  becomes inactive it will simply be ignored and any
                        #  trades by that point would have already been found)
                        flipped_df = flipped_df.loc[start:idx]
                    signal_a = flipped_df[(flipped_df['V'] < df.loc[i,'V']) & (flipped_df['Bar'] == 'up')]
                    if len(signal_a) > 0:
                        # in case of multiple signals, get lowest price for buy 
                        sell_price = max(signal_a['O'])
                        idx = signal_a['O'].idxmax()  
                        date = df.loc[idx, 'DT']
                        
                        # plot 
                        trade_entries.append(buy_price)
                        dates.append(date)
                        names.append('sell')            
                        plot_buy_flipx.append(df.loc[start, 'DT'])
                        plot_buy_flipx.append(df.loc[end, 'DT'])
                        plot_buy_flipx.append(df.loc[end, 'DT'])
                        plot_buy_flipx.append(df.loc[start, 'DT'])
                        plot_buy_flipx.append(df.loc[start, 'DT'])
                        plot_buy_flipx.append(None)
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(df.loc[i,'TZ_H'])
                        plot_buy_flipy.append(df.loc[i,'TZ_H'])
                        plot_buy_flipy.append(df.loc[i,'TZ_L'])
                        plot_buy_flipy.append(None)



#### final invalidation of trade zones, by expiry
# if current bar in df is past TZ_End, zone is expired
# df['TZ_Active'][df.index.loc[-1] > df['TZ_End'] = False    # uncomment for live data



fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])])

fig.add_trace(go.Scatter(x=plot_buyx, y=plot_buyy, fill="toself", fillcolor='green', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_buy_flipy, fill="toself", fillcolor='rgba(0, 100, 0, 1)', opacity=.4))
fig.add_trace(go.Scatter(x=plot_sellx, y=plot_selly, fill="toself", fillcolor='red', opacity=.4))
fig.add_trace(go.Scatter(x=plot_buy_flipx, y=plot_sell_flipy, fill="toself", fillcolor='rgba(100, 0, 0, 1)', opacity=.4))


fig.add_trace(go.Scatter(
    x=dates,
    y=trade_entries,
    mode="markers+text",
    name="Swing Ratings",
    text=names,
    textposition="top center",
    # fillcolor='blue'
    marker_size=10,
    marker_color='rgba(0, 0, 0, 1)'
    # yaxis= 'y2'
))

fig.update_layout(
    autosize=False,
    width=1300,
    height=1000,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    # remove "xaxis" to show weekend gaps
    xaxis = dict(  
                type="category"),
    paper_bgcolor="LightSteelBlue",  
)

fig.update(layout_xaxis_rangeslider_visible=False)


# Confidence Ratings

Visual plot: S/R and trade zones

In [0]:
####    S/R Zones    #####

# save the zones for charting 
plotx = [] # valid zones
ploty = []
plotxx = [] # invalidated zones
plotyy = []

for i in range(0, len(df)):
  swing = df.loc[i, 'Sw_Rating']

  # get the size of price zone around a swing
  # to change size adjust ADR denominator
  price_box = round(swing * df.loc[i,'ADR']/200, 4) 
  # upper price
  upper = df.loc[i,'Sw_Price'] + price_box / 2
  # lower price
  lower = df.loc[i,'Sw_Price'] - price_box / 2

  # get the time slice 
  end = swing * 10 + i # n bars forward from current bar

  # needs to be done:
  # 
  # if there is a prior swing next to the main one, and if the 
  # zone can connect to it without being intersected by price,
  # set the zone start to be the preceeding swing while maintaining
  # the Sw_Rating of the larger swing
  

  # find all swing prices that fall within those boundaries
  temp = df.loc[i:end,['Sw_Price','Sw_Size','DT']]
  temp = temp[(temp['Sw_Price'] > lower) & (temp['Sw_Price'] < upper)]

  # if there's a zone
  if len(temp) > 1:
    

    # if swing high
    if temp.reset_index().iloc[0,2] < 0:
      hi_1 = temp.reset_index().loc[0, 'Sw_Price']
      hi_2 = temp.reset_index().loc[1, 'Sw_Price']
      expired = False
      # used for setting zone extensions
      diff = pd.to_datetime(max(temp['DT'])) - pd.to_datetime(min(temp['DT']))

      ### these next lines will determine if a zone has been invalidated ###
      # look for a new high above zone
      new_high = df.loc[i:][df.loc[i:, 'H'] > upper]

      # if found 
      if len(new_high) > 1:
        # look forwards for a new low below zone
        idx = new_high.index[0]+1
        new_low = df.loc[idx:][df.loc[idx:, 'L'] < lower]
        # if found
        if len(new_low) > 1:
          # zone is invalidated  -- will plot a certain color and terminate
          # on the chart at the point of the candle that broke it
          plotxx.append(min(temp['DT']))
          plotxx.append(min(new_low['DT'])) # first low that breaches 
          plotxx.append(min(new_low['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(None)
          # full zone
          plotyy.append(lower)
          plotyy.append(lower)
          plotyy.append(upper)
          plotyy.append(upper)
          plotyy.append(lower)
          plotyy.append(None)
          expired = True
      # otherwise plot a regular extended zone
      if not expired:
        plotx.append(min(temp['DT'])) 
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(min(temp['DT']))        
        plotx.append(min(temp['DT']))
        plotx.append(None)
        ploty.append(lower)
        ploty.append(lower)
        ploty.append(upper)        
        ploty.append(upper)
        ploty.append(lower)
        ploty.append(None)



    # if swing low
    if temp.reset_index().iloc[0,2] > 0:
      low_1 = temp.reset_index().loc[0, 'Sw_Price']
      low_2 = temp.reset_index().loc[1, 'Sw_Price']
      expired = False
      # used for setting zone extensions (not currently working)
      diff = pd.to_datetime(max(temp['DT'])) - pd.to_datetime(min(temp['DT']))

      # look for a new low below zone
      new_low = df.loc[i:][df.loc[i:, 'L'] < lower]

      # if found 
      if len(new_low) > 1:
        # look forwards for a new high above zone
        idx = new_low.index[0]+1
        new_high = df.loc[idx:][df.loc[idx:, 'H'] > upper]
        # if found
        if len(new_high) > 1:

          # zone is invalidated  -- will plot a certain color and terminate
          # on the chart at the point of the candle that broke it
          plotxx.append(min(temp['DT']))
          plotxx.append(min(new_high['DT'])) # first low that breaches 
          plotxx.append(min(new_high['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(min(temp['DT']))
          plotxx.append(None)
          # full zone
          plotyy.append(lower)
          plotyy.append(lower)
          plotyy.append(upper)
          plotyy.append(upper)
          plotyy.append(lower)
          plotyy.append(None)
          expired = True

      # otherwise plot a regular extended zone
      if not expired:
        plotx.append(min(temp['DT']))
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(df.loc[:-1,'DT'])
        plotx.append(min(temp['DT']))        
        plotx.append(min(temp['DT']))
        plotx.append(None)
        ploty.append(lower)
        ploty.append(lower)
        ploty.append(upper)        
        ploty.append(upper)
        ploty.append(lower)
        ploty.append(None)

    # # return sum and count of all swing ratings within window
    # df.loc[i,'SR_Sum'] = sum(df.loc[start:end,'Sw_Rating'])
    # df.loc[i, 'SR_Count'] = len(temp)
    # # print(temp)
